# Query News Crawler

## 0. imports

In [1]:
%load_ext lab_black

In [2]:
import sys

sys.path.append("..")

In [3]:
import os
import re
import time
import random
import dill
import requests
import urllib
import bs4
import newspaper

from collections import defaultdict
from datetime import datetime, timedelta
from typing import List, Dict, Union

from bs4 import BeautifulSoup
from bs4.element import NavigableString
from kss import split_sentences

from newspaper import Article
from tqdm import tqdm, trange
from crawler.utils import clean_text

## 1. QueryNewsCrawler class

### 1.1 line by line coding

In [6]:
# query = "코스피"
# query = urllib.parse.quote(query)

# main_url = "https://search.naver.com/search.naver"
# query_url = "?where=news&sm=tab_pge&query="
# # 네이버 뉴스의 경우 page가 아닌
# # 뉴스 기사 개수 카운팅으로 보여주고 있음
# # 1 ~ 4000개까지 10개 단위로 보여줌
# page_url = "&sort=0&photo=0&field=0&pd=0&ds=&de=&cluster_rank=23&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:all,a:all&start="

# start = 1
# start_range = list(range(1, 4000, 10))
# url = f"{main_url}{query_url}{query}{page_url}{start}"

In [5]:
# req = requests.get(url)
# soup = BeautifulSoup(req.text, "html.parser")

# # ul lists
# ul_lists = soup.find("ul", {"class": "list_news"})
# # news links
# links = ul_lists.find_all("a", {"class": "news_tit"})
# link_title_dict = {row["href"]: {"title": row["title"]} for row in links}


# # update news texts
# news_texts = ul_lists.find_all("a", {"class": "api_txt_lines dsc_txt_wrap"})
# for row in news_texts:
#     link_title_dict[row["href"]]["text"] = row.text

In [7]:
# # request and soup
# req = requests.get(url)
# soup = BeautifulSoup(req.text, "html.parser")

# # ul lists and li lists
# ul_lists = soup.find("ul", {"class": "list_news"})
# li_lists = ul_lists.findChildren("li", {"class": "bx"})

# news_data = []
# for li in li_lists:

#     # news link & title
#     link = li.find("a", {"class": "news_tit"})
#     href, title = link["href"], link["title"]

#     # news text
#     text = li.find("a", {"class": "api_txt_lines dsc_txt_wrap"}).text

#     # thumb nail if exist
#     thumb = ""
#     thumb_link = li.find("img", {"class": "thumb api_get"})
#     if thumb_link:
#         thumb = thumb_link["src"]

#     # update link_title_dict
#     news_data.append({"href": href, "title": title, "text": text, "thumb": thumb})

In [8]:
# li = li_lists[3]
# li.find_all("a", {"class": "info"})
# # li.find("span", {"class": "info"}).text

In [10]:
# # 뉴스 링크
# links = li.find_all("a", {"class": "info"})
# naver_news_url = [link for link in links if link.text == "네이버뉴스"]
# if naver_news_url:
#     naver_news_url = naver_news_url[0]
#     naver_news_url = naver_news_url["href"].replace("&amp;", "&")
# else:
#     naver_news_url = None

In [11]:
# links = li.find_all("a", {"class": "info"})
# naver_news_url = [link for link in links if link.text == "네이버뉴스"][0]
# naver_news_url = naver_news_url["href"].replace("&amp;", "&")

# # 언어가 한국어이므로 language='ko'로 설정
# a = Article(naver_news_url, language="ko")
# a.download()
# a.parse()

In [12]:
# for news in news_data:
#     href = news["href"]

#     # 언어가 한국어이므로 language='ko'로 설정
#     a = Article(href, language="ko")
#     a.download()
#     a.parse()

#     news["date"] = a.publish_date

In [13]:
# news_data

### 1.2 Class

In [25]:
class QueryNewsCrawler:
    def __init__(self):
        # 네이버 뉴스의 경우 page가 아닌
        # 뉴스 기사 개수 카운팅으로 보여주고 있음
        # 1 ~ 4000개까지 10개 단위로 보여줌
        self.main_url = "https://search.naver.com/search.naver"
        self.query_url = "?where=news&sm=tab_pge&query="
        self.page_url = "&sort=0&photo=0&field=0&pd=0&ds=&de=&cluster_rank=23&mynews=0&office_type=0&"
        self.page_url += (
            "office_section_code=0&news_office_checked=&nso=so:r,p:all,a:all&start="
        )

    def crawl_news_by_query(self, query: str, count: int = 200) -> List[Dict]:
        self.query = query
        parsed_query = urllib.parse.quote(query)

        if count:
            start_range = list(range(1, count, 10))
        else:
            start_range = list(range(1, 4000, 10))

        news_data = []
        for s_idx in tqdm(start_range):
            url = f"{self.main_url}{self.query_url}{parsed_query}{self.page_url}{s_idx}"

            # request and soup
            req = requests.get(url)
            soup = BeautifulSoup(req.text, "html.parser")

            # ul lists and li lists
            ul_lists = soup.find("ul", {"class": "list_news"})
            li_lists = ul_lists.findChildren("li", {"class": "bx"})
            for li in li_lists:
                # news link & title
                link = li.find("a", {"class": "news_tit"})
                href, title = link["href"], link["title"]

                # newspaper3k instance
                try:
                    news_instance = Article(href, language="ko")
                    news_instance.download()
                    news_instance.parse()
                except:
                    news_instance = None

                # 언론사
                press = li.find("a", {"class": "info press"}).text

                # publish date
                publish_date = self._get_datetime(news_instance, li)

                # news short_text
                short_text = li.find("a", {"class": "api_txt_lines dsc_txt_wrap"}).text

                # news text and top_image
                text, top_iamge = self._get_article(news_instance, li)

                # thumb nail if exist
                thumb = ""
                thumb_link = li.find("img", {"class": "thumb api_get"})
                if thumb_link:
                    thumb = thumb_link["src"]

                # update link_title_dict
                news_data.append(
                    {
                        "href": href,
                        "date": publish_date,
                        "press": press,
                        "title": title,
                        "short_text": short_text,
                        "text": text,
                        "top_image": top_iamge,
                        "thumb": thumb,
                    }
                )
            time.sleep(random.uniform(0.6, 0.9))
        return news_data

    def _get_article(
        self, article: newspaper.article.Article, li: bs4.element.Tag
    ) -> str:
        text, top_image = None, None
        if article is not None:
            text = article.text
            top_img = article.top_image

        links = li.find_all("a", {"class": "info"})
        naver_news_url = [link for link in links if link.text == "네이버뉴스"]
        if naver_news_url:
            naver_news_url = naver_news_url[0]
            naver_news_url = naver_news_url["href"].replace("&amp;", "&")
        else:
            naver_news_url = None

        if text is None and naver_news_url is not None:
            try:
                a = Article(naver_news_url, language="ko")
                a.download()
                a.parse()

                text = a.text
            except:
                pass
        elif top_image is None and naver_news_url is not None:
            try:
                a = Article(naver_news_url, language="ko")
                a.download()
                a.parse()

                top_image = a.top_image
            except:
                pass
        return text, top_image

    def _get_datetime(
        self,
        article: newspaper.article.Article,
        li: bs4.element.Tag,
    ) -> datetime:
        publish_date = None
        if article is not None:
            publish_date = article.publish_date

        if publish_date is None:
            date_list = li.find_all("span", {"class": "info"})
            date = None
            if len(date_list) > 1:
                date = date_list[-1].text
            else:
                date = li.find("span", {"class": "info"}).text
                date = date.split()[0]

            if "분" in date:
                minutes = re.sub(r"[^\d+]", "", date)
                publish_date = datetime.now() - timedelta(minutes=int(minutes))
            elif "시간" in date:
                hours = re.sub(r"[^\d+]", "", date)
                publish_date = datetime.now() - timedelta(hours=int(hours))
            elif "일" in date:
                days = re.sub(r"[^\d+]", "", date)
                publish_date = datetime.now() - timedelta(days=int(days))
            else:
                try:
                    publish_date = datetime.strptime(date, "%Y.%m.%d.")
                except:
                    pass
        return publish_date

## 2. Test

In [19]:
crawler = QueryNewsCrawler()

In [23]:
query = "코스피"
count = 10
news_data = crawler.crawl_news_by_query(query=query, count=count)

100%|██████████| 1/1 [00:08<00:00,  8.68s/it]


In [26]:
news_data[0]

{'href': 'https://www.hankyung.com/finance/article/2022031963016',
 'date': datetime.datetime(2022, 3, 20, 8, 0, 5, tzinfo=tzoffset(None, 32400)),
 'press': '한국경제언론사 선정',
 'title': '악재에 내성 생긴 코스피…"낙폭과대·내수株 주목" [주간전망]',
 'short_text': '코스피가 반등을 모색하고 있다. 아직 증시를 둘러싼 악재의 해소 국면으로 접어들었다고 보기는 어렵지만... 20일 한국거래소에 따르면 지난주(14~18일) 코스피는 1.72% 오른 2707.02에 거래를 마쳤다. 이 기간 동안...',
 'text': '글로벌증시, 유가 100달러 재돌파에도 강세 유지\n\nNH투자증권, 예상밴드로 2650~2800 제시\n\n“업종·종목별 순환매 가능성”\n\n사진=연합뉴스\n\n코스피가 반등을 모색하고 있다. 아직 증시를 둘러싼 악재의 해소 국면으로 접어들었다고 보기는 어렵지만, 증시가 우크라이나 전쟁, 원자재 가격 급등, 통화 긴축에 따른 경기 위축 등 우려스러운 이슈에 대한 내성을 확보한 모습이다.증권가도 증시의 하방 압력보다는 상승 여력에 주목할 때라는 분석에 힘을 실으며, 펀더멘털에 주목하라고 조언한다.20일 한국거래소에 따르면 지난주(14~18일) 코스피는 1.72% 오른 2707.02에 거래를 마쳤다.이 기간 동안 외국인이 유가증권시장에서 1조896억원 어치의 주식을 팔았다. 일별로는 지난 17일 하루를 제외하고 모두 순매도했다. 외국인의 매도세는 지난 4일 이후 8거래일동안 이어졌고, 하루 순매수한 뒤 18일에는 다시 1030억원 어치를 팔았다.기관과 개인은 각각 6549억원 어치와 4272억원 어치를 순매수했다.주초에는 중국에서 코로나19 오미크론 변이의 확산으로 4대 1선도시 중 하나인 선전시를 일주일동안 봉쇄하기로 했다는 소식이 증시를 짓눌렀다.반등의 계기는 국제유가의 급락이었다. 뉴욕상업거래